### Multiscale Spatiotemporal Vision Transformer (MST-ViT | PixFormer)

### Misael M. Morales, 2024
***

In [1]:
from main import *

hete = Heterogeneity()


------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.1.0+cu121 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3080
------------------------------------------------------------



In [ ]:
hete.make_dataloaders()

In [ ]:
hete.trainer()

In [ ]:
hete.tester()

In [ ]:
hete.plot_losses()

***
# END